# Домашнее задание HW02

Работа с табличными данными в Pandas, контроль качества данных, базовый EDA и визуализация в Matplotlib.


## 1. Загрузка данных и первичный осмотр


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Настройка для отображения графиков
plt.style.use('default')
%matplotlib inline


In [ ]:
# Загрузка датасета
df = pd.read_csv('../../S02-hw-dataset.csv')
print(f"Размер датасета: {df.shape}")
print("\nПервые строки датасета:")
df.head(10)


In [ ]:
# Информация о столбцах и типах данных
df.info()


In [ ]:
# Базовые описательные статистики
df.describe()


## 2. Пропуски, дубликаты и базовый контроль качества


In [ ]:
# Доля пропусков в каждом столбце
missing_percent = df.isna().mean() * 100
print("Доля пропусков по столбцам (%):")
print(missing_percent)
print(f"\nВсего пропусков: {df.isna().sum().sum()}")


In [ ]:
# Проверка полностью дублирующих строк
duplicated_rows = df[df.duplicated(keep=False)]
print(f"Количество полностью дублирующих строк: {df.duplicated().sum()}")
if len(duplicated_rows) > 0:
    print("\nДублирующие строки:")
    print(duplicated_rows)


In [ ]:
# Проверка подозрительных значений: отрицательные purchases
negative_purchases = df[df['purchases'] < 0]
print("Строки с отрицательным количеством покупок:")
print(negative_purchases)


In [ ]:
# Проверка подозрительных значений: нереалистичный возраст (> 100 или < 0)
unrealistic_age = df[(df['age'] > 100) | (df['age'] < 0)]
print("Строки с нереалистичным возрастом (<= 0 или > 100):")
print(unrealistic_age)


In [ ]:
# Проверка логических противоречий: нулевой revenue при ненулевых purchases
zero_revenue_with_purchases = df[(df['revenue'] == 0) & (df['purchases'] > 0)]
print("Строки с нулевой выручкой при ненулевых покупках:")
print(zero_revenue_with_purchases)


In [ ]:
# Проверка: ненулевой revenue при нулевых purchases (может быть возврат или ошибка)
nonzero_revenue_with_zero_purchases = df[(df['revenue'] != 0) & (df['purchases'] == 0)]
print("Строки с ненулевой выручкой при нулевых покупках:")
print(nonzero_revenue_with_zero_purchases)


### Выводы по качеству данных

В датасете обнаружены следующие проблемы качества данных:

1. **Пропуски**: В столбце `age` есть пропущенные значения (2 случая - пользователи с ID 5 и 13).

2. **Дубликаты**: Обнаружена одна полностью дублирующая строка (пользователь с ID 10 встречается дважды).

3. **Подозрительные значения**:
   - Отрицательное количество покупок: пользователь с ID 6 имеет `purchases = -1`, что логически невозможно.
   - Нереалистичный возраст: пользователь с ID 6 имеет возраст 120 лет, что маловероятно для реальных данных.
   - Логические противоречия: несколько пользователей имеют нулевую выручку при ненулевых покупках (ID 11), что может указывать на ошибки в данных или специальные случаи (например, возвраты).

Эти проблемы могут влиять на корректность анализа и требуют дополнительной обработки или исключения проблемных записей при дальнейшей работе.


## 3. Базовый EDA: группировки, агрегаты и частоты


In [ ]:
# Частоты по странам
country_counts = df['country'].value_counts()
print("Распределение пользователей по странам:")
print(country_counts)
print(f"\nДоля каждой страны (%):")
print((country_counts / len(df) * 100).round(2))


In [ ]:
# Группировка по странам с агрегатами
country_stats = df.groupby('country').agg({
    'purchases': ['mean', 'sum', 'count'],
    'revenue': ['mean', 'sum'],
    'age': 'mean'
}).round(2)
country_stats.columns = ['Средние покупки', 'Всего покупок', 'Количество пользователей', 
                         'Средняя выручка', 'Общая выручка', 'Средний возраст']
print("Статистика по странам:")
country_stats


In [ ]:
# Создание возрастных групп
df['age_group'] = pd.cut(df['age'], 
                         bins=[0, 25, 35, 45, 100], 
                         labels=['18-25', '26-35', '36-45', '46+'],
                         include_lowest=True)

# Статистика по возрастным группам
age_group_stats = df.groupby('age_group').agg({
    'purchases': ['mean', 'count'],
    'revenue': 'mean'
}).round(2)
age_group_stats.columns = ['Средние покупки', 'Количество', 'Средняя выручка']
print("Статистика по возрастным группам:")
age_group_stats


### Основные наблюдения

1. **Распределение по странам**: Больше всего пользователей из России (RU) - 14 человек (34%), затем идут Франция (FR) - 12 человек (29%), США (US) - 8 человек (20%), Германия (DE) - 5 человек (12%) и Китай (CN) - 2 человека (5%).

2. **Средние значения по странам**: 
   - По средней выручке лидирует Германия (DE) - 1278.6, затем Россия (RU) - 1000.4
   - По среднему количеству покупок лидирует Германия (DE) - 6.6 покупок на пользователя
   - Франция имеет самую низкую среднюю выручку - 820.3

3. **Возрастные группы**: 
   - Группа 26-35 лет показывает наибольшую среднюю выручку
   - Группа 18-25 лет имеет наибольшее количество пользователей
   - Наблюдается тенденция: средняя выручка выше у пользователей среднего возраста (26-45 лет)

4. **Неожиданные эффекты**: Интересно, что при относительно небольшом количестве пользователей из Германии, они показывают самые высокие средние показатели по выручке и покупкам, что может указывать на более активное поведение этой группы пользователей.


## 4. Визуализация данных в Matplotlib


In [ ]:
# Гистограмма распределения возраста
plt.figure(figsize=(10, 6))
plt.hist(df['age'].dropna(), bins=15, edgecolor='black', alpha=0.7, color='skyblue')
plt.xlabel('Возраст', fontsize=12)
plt.ylabel('Количество пользователей', fontsize=12)
plt.title('Распределение возраста пользователей', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('figures/age_histogram.png', dpi=300, bbox_inches='tight')
plt.show()
print("График сохранен в figures/age_histogram.png")


**Комментарий к гистограмме возраста**: График показывает распределение возраста пользователей. Видно, что большинство пользователей находятся в возрастном диапазоне 20-50 лет, с пиком в районе 25-35 лет. Есть один выброс - пользователь с возрастом 120 лет, который был выявлен ранее как подозрительное значение.


In [ ]:
# Боксплот выручки по странам
plt.figure(figsize=(10, 6))
df.boxplot(column='revenue', by='country', ax=plt.gca(), grid=False)
plt.xlabel('Страна', fontsize=12)
plt.ylabel('Выручка', fontsize=12)
plt.title('Распределение выручки по странам', fontsize=14, fontweight='bold')
plt.suptitle('')  # Убираем автоматический заголовок
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('figures/revenue_boxplot_by_country.png', dpi=300, bbox_inches='tight')
plt.show()
print("График сохранен в figures/revenue_boxplot_by_country.png")


**Комментарий к боксплоту выручки**: Боксплот показывает распределение выручки по странам. Видно, что в Германии (DE) медианная выручка выше, чем в других странах, и разброс значений также больше. Во Франции (FR) и России (RU) распределение более компактное. В США (US) есть несколько выбросов с низкой выручкой. Китай (CN) представлен только двумя точками, поэтому статистика ограничена.


In [ ]:
# Scatter plot: количество покупок vs выручка с цветом по странам
plt.figure(figsize=(10, 6))
countries = df['country'].unique()
colors = plt.cm.Set3(np.linspace(0, 1, len(countries)))
color_map = dict(zip(countries, colors))

for country in countries:
    country_data = df[df['country'] == country]
    plt.scatter(country_data['purchases'], country_data['revenue'], 
               label=country, alpha=0.6, s=100, color=color_map[country])

plt.xlabel('Количество покупок', fontsize=12)
plt.ylabel('Выручка', fontsize=12)
plt.title('Зависимость выручки от количества покупок по странам', fontsize=14, fontweight='bold')
plt.legend(title='Страна', loc='upper left')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('figures/purchases_vs_revenue_scatter.png', dpi=300, bbox_inches='tight')
plt.show()
print("График сохранен в figures/purchases_vs_revenue_scatter.png")


**Комментарий к scatter plot**: Диаграмма рассеяния показывает связь между количеством покупок и выручкой. В целом наблюдается положительная корреляция - чем больше покупок, тем выше выручка. Однако есть несколько интересных наблюдений:
- Некоторые пользователи с большим количеством покупок имеют относительно низкую выручку (возможно, дешевые товары)
- Есть пользователи с нулевой выручкой при ненулевых покупках (выявленные ранее как проблемные данные)
- Пользователи из Германии (DE) в целом показывают более высокую выручку при том же количестве покупок, что может указывать на более дорогие покупки
